In [49]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
# from matplotlib.pyplot import rc
import csv
import math
from collections import defaultdict

In [84]:
def parse_ncu_csv(fname):
    df = pd.read_csv(fname)
    df.columns = [c.replace(' ', '_') for c in df.columns]
    df1 = df[ (df.Metric_Name == 'sm__cycles_active.avg') | (df.Metric_Name == 'sm__inst_executed.avg') | (df.Metric_Name == 'sm__inst_executed_pipe_tensor.avg') | (df.Metric_Name == 'sm__pipe_tensor_cycles_active.avg')
    | (df.Metric_Name == 'Memory [%]') | (df.Metric_Name == 'SM [%]') | (df.Metric_Name == 'SM Active Cycles')]

    df1 = df1[['ID', 'Kernel_Name','Metric_Name', 'Metric_Value']]
    df1.head(50)

    profile_res = defaultdict(lambda: defaultdict(int))
    for index, row in df1.iterrows():
        profile_res[row['Kernel_Name']][row['Metric_Name']] += float(row['Metric_Value'].replace(',', ''))
        if row['Metric_Name'] == 'Memory [%]':
            profile_res[row['Kernel_Name']]['count'] += 1
        elif row['Metric_Name'] != 'SM [%]' :
            profile_res['Total'][row['Metric_Name']] += float(row['Metric_Value'].replace(',', ''))
    for c in profile_res: 
        if c != 'Total':
            profile_res[c]['SM [%]'] /= profile_res[c]['count']
            profile_res[c]['Memory [%]'] /= profile_res[c]['count']

    # for c in profile_res: 
    #     print(c,profile_res[c],'\n')
    return profile_res

In [98]:
def process_result(app_name, output_csv):
    prof_res = parse_ncu_csv(app_name + '.csv')
    comma_sep = ',,,,,,,\n'
    output_csv.write(app_name+comma_sep)
    def write_res(output_csv,c):
        output_csv.write(c.replace(',',';')+','+
                            str(prof_res[c]['Memory [%]'])+','+
                            str(prof_res[c]['SM [%]'])+','+
                            str(prof_res[c]['SM Active Cycles'])+','+
                            str(prof_res[c]['sm__cycles_active.avg'])+','+
                            str(prof_res[c]['sm__inst_executed.avg'])+','+
                            str(prof_res[c]['sm__pipe_tensor_cycles_active.avg'])+','+
                            str(prof_res[c]['sm__inst_executed_pipe_tensor.avg'])+',\n'
                            )
    for c in prof_res:
        if c != 'Total':
            write_res(output_csv,c)
    write_res(output_csv,'Total')


In [107]:
output_csv = open('profile_result.csv','w')
# write header
output_csv.write('Kernel_Name,Memory [%],SM [%],SM Active Cycles,SM CYCLE COUNT,SM INSTR COUNT,SM TENSOR CYCLE COUNT,SM TENSOR INSTR COUNT,\n')

tgt_app = ['profile_result_apsp_baseline',
'profile_result_apsp',
'profile_result_aplp_baseline',
'profile_result_aplp',
'profile_result_mcp_baseline',  
'profile_result_mcp',
'profile_result_maxrp_baseline',
'profile_result_maxrp',
'profile_result_minrp_baseline',
'profile_result_minrp',                                   
'profile_result_mst_baseline',
'profile_result_mst',
'profile_result_gtc_baseline',
'profile_result_gtc',
'profile_result_pld_baseline',
'profile_result_pld']

tgt_app = ['profile_result_microbench']
output_csv = open('profile_result_mb.csv','w')

for t in tgt_app:
    process_result(t,output_csv)
  

output_csv.close()

In [105]:
pd.unique(df['Metric_Name'])

array(['sm__cycles_active.avg', 'sm__inst_executed.avg',
       'sm__inst_executed_pipe_tensor.avg',
       'sm__pipe_tensor_cycles_active.avg', 'DRAM Frequency',
       'SM Frequency', 'Elapsed Cycles', 'Memory [%]', 'SOL DRAM',
       'Duration', 'SOL L1/TEX Cache', 'SOL L2 Cache', 'SM Active Cycles',
       'SM [%]', nan], dtype=object)